In [140]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split 
from sklearn.metrics import r2_score, recall_score, precision_score, accuracy_score
import numpy as np

from sklearn.ensemble import IsolationForest
from joblib import dump, load
import pandas as pd 
import plotly.express as px

In [141]:
# print(sklearn.__version__) # 0.24.1
# print(numpy.__version__) #1.20.1
# print(pandas.__version__) #1.2.4
# print(plotly.__version__) #5.1.0


In [142]:
dfo = pd.read_csv("..\\Data\\SmA-Four-Tank-Batch-Process_V2.csv",sep=';')
dfo['timestamp']=pd.to_datetime(dfo['timestamp'])
#df_day=dfo.copy()
#df_day['h']=df_day['timestamp'].dt.hour

In [143]:
df=dfo.copy()

df=df.drop(columns=["timestamp","CuStepNo ValueY"])
c=list(df.columns)
c[0]="Dev"
df.columns=c
df=df[df['Dev']!=0]
df['Dev']=df['Dev']!=1
df['Dev']=df['Dev'].astype('int')
df_clean=df.copy()

In [144]:
df_clean.to_csv('..\\Data\\data_01_clean.csv')
df_clean=df_clean.sample(250000)
X_train,X_test,y_train,y_test=train_test_split(df_clean.drop(columns=['Dev']),df_clean['Dev'],shuffle=False,random_state=42)

In [145]:
model=RandomForestClassifier(n_estimators=100)
model.fit(X_train,y_train)
dump(model,"rf.pkl")

['rf.pkl']

# Metriken der Trainings-Daten

In [146]:
y_pred_train=model.predict(X_train)
print(f'R2:{r2_score(y_train,y_pred_train)}\nAccuracy: {accuracy_score(y_train,y_pred_train)}\nRecall: {recall_score(y_train,y_pred_train)}\nPrecision: {precision_score(y_train,y_pred_train)}')

R2:1.0
Accuracy: 1.0
Recall: 1.0
Precision: 1.0


In [147]:

tn, fp, fn, tp = confusion_matrix(y_train,y_pred_train).ravel()
print(f'TN: {tn}')
print(f'FP: {fp}')
print(f'FN: {fn}')
print(f'TP: {tp}')

TN: 25070
FP: 0
FN: 0
TP: 162430


# Metriken der Test-Daten

In [148]:
y_pred=model.predict(X_test)
print(f'R2:{r2_score(y_test,y_pred)}\nAccuracy: {accuracy_score(y_test,y_pred)}\nRecall: {recall_score(y_test,y_pred)}\nPrecision: {precision_score(y_test,y_pred)}')

R2:0.8518652212817277
Accuracy: 0.982928
Recall: 0.992286826711937
Precision: 0.9880751139203293


In [149]:
y_pred=model.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test,y_pred).ravel()
print(f'TN: {tn}')
print(f'FP: {fp}')
print(f'FN: {fn}')
print(f'TP: {tp}')

TN: 7658
FP: 649
FN: 418
TP: 53775


Hinweis: <br> 
Durch die Ergänzung der Vorhersage durch eine zeitliche Klassifizierung (Dargestellt in "zeitliche_Klassifizierung.ipynb"), können dann alle fehlerhaften Batches als fehlerhaft klassifiziert werden. <br> 
Dies müsste in einem produktiven System mit live Daten in einer Software umgesetzt werden. <br>